**Connect to Elastic Search**

In [1]:
from pprint import pprint 
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

client_info = es.info()
print("Connected to Elasticsearch!")
pprint(client_info.body)

Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'T1HeaWnRTOqX_BBgREVVbA',
 'name': '64c49e436740',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-10-21T10:06:21.288851013Z',
             'build_flavor': 'default',
             'build_hash': '25d88452371273dd27356c98598287b669a03eae',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '10.3.1',
             'minimum_index_compatibility_version': '8.0.0',
             'minimum_wire_compatibility_version': '8.19.0',
             'number': '9.2.0'}}


**Create Index with Auto Mappings**

In [3]:
es.indices.delete(index="my_index", ignore_unavailable=True)
es.indices.create(index="my_index_1") 
es.indices.create(index="my_index_2")

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index_2'})

In [4]:
import json 
from tqdm import tqdm

documents = json.load(open("data.json"))

for document in tqdm(documents, total=len(documents)):
    es.index(index="my_index_1", body=document)
    es.index(index="my_index_2", body=document) 

100%|██████████| 5/5 [00:01<00:00,  3.91it/s]


**Search**

In [7]:
response = es.search(
    index="my_index_1", 
    body={
        "query": {"match_all": {}}
    }
)

print(f"Found {response["hits"]["total"]["value"]} documents in my_index_1.")

Found 5 documents in my_index_1.


In [8]:
response = es.search(
    index="my_index_2", 
    body={
        "query": {"match_all": {}}
    }
)

print(f"Found {response["hits"]["total"]["value"]} documents in my_index_2.")

Found 5 documents in my_index_2.


**Search both indexes together**

In [9]:
response = es.search(
    index="my_index_1,my_index_2",
    body={
        "query": {"match_all": {}}
    }
)

print(f"Found {response["hits"]["total"]["value"]} documents in my_index_1 & my_index_2.")

Found 10 documents in my_index_1 & my_index_2.


User wild card *

In [10]:
response = es.search(
    index="my_index*", 
    body={
        "query": {"match_all": {}}
    }
)

print(f"Found {response["hits"]["total"]["value"]} documents in all the indexes start with my_index prefix.")

Found 10 documents in all the indexes start with my_index prefix.


If you want to search all the available indexes then you can use _all

In [11]:
response = es.search(
    index="_all", 
    body={
        "query": {"match_all": {}}
    }
)

print(f"Found {response["hits"]["total"]["value"]} documents in all the indexes.")

Found 10 documents in all the indexes.
